# Bus Model Information
This notebook is intended to act as a location where values and sources for different bus, trip, and ESS models are stored. 

In [13]:
# General Purpose:
import sys
sys.path.append("../src/")
from reRoute_Dynamics import Object_Params as op
bus_directory = "./KC_Example_Data/Saved_Objects/Busses/"
trip_directory =  "./KC_Example_Data/Saved_Objects/Trips/"
ESS_directory =  "./KC_Example_Data/Saved_Objects/ESSes/"

a_prof_path = "KC_Example_Data/Acceleration_Profiles/Braunschweig_Acceleration.csv"#Interpolated from @NREL_Drive_Cylces

In [14]:
base_trip = op.Trip(pass_mass = 84, #kg, calculated from @CDC_Human_Mass
                    limit_MOE = .15, # percent of the speed limit the value can be within. 
                    signal_rest = 65/2, # seconds, based on @WSDOT_Signals
                    signal_chance = .541666, # fraction o fthe time the bus will hit a red light when passing a signal. This is calculated based on @WSDOT_Signals
                    stop_rest = 7, # seconds per passenger boarding. This is a Guess. 
                    sign_rest = 7, # seconds. This is a guess of how long a bus will stop at a stop sign.
                    end_rest = 10, # seconds.amount of time the bus rests at the beginning or end of a route before bus is turned off.
                    air_density = 1.22, # based on seattle typical temperature of 12.7 degrees C. @Air_Density
                    wind_speed = 1.78, #m/s, based on typical wind speed in seattle @Weatherspark_Seattle
                    wind_heading = 'SE', #per @Weatherspark_Seattle
                    temperature = 12.7, #deg C, per @Weatherspark_Seattle
                    interp_length = 10, # meters, guess. 
                    mean_ridership = 3.5, # People. Back of napkin from @KC_Rider_Stats and other KCM route data
                    seed = 42, # random seed, subject to change
                    lg=43, #savgol param
                    deg = 3, #savgol param
                    stop_margin = 1 #m/s, margin for the velocity to be considered 'stopped'
                   )
base_trip.save(trip_directory+"base_seattle_trip.txt")

'./KC_Example_Data/Saved_Objects/Trips/base_seattle_trip.txt'

In [15]:
# XDE60 HEV version: 
XDE60 = op.Bus(bus_mass = 19096, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.3, #m, @NF_Xcelsior
               drag_coeff = .7, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XDE60.save(bus_directory + "XDE60.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE60_ESS = op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .97, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .6, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000 #w, finally, some good sourcable values.@XDE35_Manuals
                   # W @Siemens_Traction_Motor
                  )
XDE60_ESS.save(ESS_directory + "XDE60_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE60_ESS.txt'

In [16]:
# XDE40/XT40 HEV version: 
XT40 =op.Bus(bus_mass = 13835, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.3, #m, @NF_Xcelsior
               drag_coeff = .7, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 246000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XT40.save(bus_directory + "XT40.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XT40_ESS =  op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .97, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .6, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000 #w, finally, some good sourcable values.@XDE35_Manual
                   # W @Siemens_Traction_Motor
                  )

XT40_ESS.save(ESS_directory + "XT40_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XT40_ESS.txt'

In [5]:
12*3.3*16

633.5999999999999

In [6]:
import ast
import random
import numpy as np
def optimizefunc(func, ranges):
    '''
    optimizefunc takes a function with n parameters, and a set of n ranges for each parameter,
    and attempts to minimize the return value of the function. 
    '''
    # Set seed for optimization randomization
    random.seed(42)
    
    # Max iterations is 30
    max_iterations=10000
    iterations = 0
    
    rep_iterations = 0
    # Get number of dimensions for hyper-triangle
    ndim = 1+ len(ranges)
    
    # Dictionary for the points
    point_dict = {}
    lowest_max = (9999, 0)
    # render the first point set, this could be done with multiprocessing
    for i in range(ndim):
        args = []
        for j in range(len(ranges)):
            #print(ranges[j])
            arg = random.uniform(*ranges[j])
            args.append(arg)
        result = func(*tuple(args))
        point_dict[str(args)]=result

    # Loop the number of times specified
    while iterations < max_iterations:
        
        print("Evaluating literals...", end='\r')
        
        # get the furthest point's parameters
        furthest_pt = ast.literal_eval(max(point_dict, key=point_dict.get))
        furthest_val = point_dict[max(point_dict, key=point_dict.get)]
        
        # Check if the lowest max value is higher than the current
        if furthest_val < lowest_max[0]:
            lowest_max = (furthest_val, furthest_pt)

        # remove it from the dict
        del point_dict[max(point_dict, key=point_dict.get)]
        
        print("Loading hyperplane params...", end='\r')
        # get the vectors for the hyperplane
        hyperplane_vectors = list(point_dict.keys())
        
        # Convert them to actual vectors
        for i in range(len(hyperplane_vectors)):
            hyperplane_vectors[i] = ast.literal_eval(hyperplane_vectors[i])
        
        print("Calculating hyperplane...", end='\r')
        # get the hyperplane equation
        hyperplane_eqn = 0
        try:
            
            hyperplane_eqn = np.linalg.solve(hyperplane_vectors, [-1]*len(hyperplane_vectors))
        except:
            print(min(point_dict, key=point_dict.get), point_dict[min(point_dict, key=point_dict.get)])
            break
        
        print("Evaluating hyperplane normal...", end='\r')
        # get the hyperplane normal
        hyperplane_normal_len = np.sqrt(sum(np.square(hyperplane_eqn)))
        
        # get the normalized hyperplane vector
        normalized_vector = hyperplane_eqn/hyperplane_normal_len
        
        print("Calculating Mirror Point...", end='\r')
        # Get the sign distance for the furthest point
        signdist = sum(normalized_vector*furthest_pt)
        
        # Get the mirrored version of that point
        mirrorpt = furthest_pt -2*normalized_vector*signdist
        
        mirrorpt = ensure_in_range(mirrorpt, ranges)
        
        print("Calculating Centroid...", end='\r')
        # Calculate the centroid of the hyperplane and mirror
        centroid = (sum([np.asarray(ls) for ls in hyperplane_vectors]) + mirrorpt)/(len(hyperplane_vectors)+1)
        centroid = ensure_in_range(centroid, ranges)
        
        print("Running Mirror Point...", end='\r')
        # run the function with the mirror and the centroid
        mir = func(*mirrorpt)
        print("Running Centroid Point...", end='\r')
        cent = func(*centroid)
        
        # if the mirror result is lower than the centroid, keep it
        if mir < cent:
            point_dict[str(mirrorpt.tolist())] = mir
            
        # otherwise, keep the centroid result
        else:
            point_dict[str(centroid.tolist())] = cent
        
        if point_dict[max(point_dict, key=point_dict.get)] == furthest_val:
            rep_iterations += 1
        
        if rep_iterations > 2: 
            closest_key = min(point_dict, key=point_dict.get)
            closest_val = point_dict[closest_key]
            point_dict = {}
            point_dict[closest_key] = closest_val
            for i in range(ndim-1):
                args = []
                for j in range(len(ranges)):
                    #print(ranges[j])
                    arg = random.uniform(*ranges[j])
                    args.append(arg)
                result = func(*tuple(args))
                point_dict[str(args)]=result
            rep_iterations = 0
        # update iterations. 
        #print(iterations, point_dict[min(point_dict, key=point_dict.get)], point_dict[max(point_dict, key=point_dict.get)])
        iterations+=1
    
    # return the lowest value parameter set found. 
    print(lowest_max)
    return min(point_dict, key=point_dict.get), point_dict[min(point_dict, key=point_dict.get)]

def ensure_in_range(point, point_bounds):
    '''
    verifies that a point in n dimensions lies within the n bounds [(min, max) * n],
    and if it isnt, set the point to be within the bounds.
    '''
    for i in range(len(point)):
        param_bounds = point_bounds[i]
        if point[i] < min(param_bounds):
            point[i] = min(param_bounds)
        elif point[i]> max(param_bounds):
            point[i] = max(param_bounds)
        else:
            point[i]=point[i]
    return point

def rsq(a, b):
    '''
    returns the r^2 value between two datasets
    '''
    # a is the raw data, b is the model
    ssres = ((a - b)**2).sum() 
    sstot = ((a-a.mean())**2).sum()
    r2 = 1 - (ssres/sstot)
    return r2

In [17]:
# XDE35 BEV version: 
def find_SP(s1, p1, s2, p2):
    XDE35 = op.Bus(bus_mass = 13920, #kg, @NF_Xcelsior
                   frontal_width = 2.6, #m,@NF_Xcelsior
                   frontal_height = 3.38, #m, @NF_Xcelsior
                   drag_coeff = .79, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
                   friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
                   braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
                   br_factor = .5, # driver braking aggression. Variable based on driver.
                   i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                                   # however, there is no substantive explanation as to how it was calculated/obtained.
                   max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
                   a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
                   max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
                   max_dt = .5, #s, timestep for that extension. 
                   max_P = 300000 # W, @Siemens_Traction_Motor 
                  )

    # Saving the object.
    XDE35.save(bus_directory + "XDE35.txt")


    # The XDE35 uses the @Siemens_Traction_Motor
    # Cooling system: @E-Fan_Cooling
    # Approx 10080 watts at max cooling
    # HVAC system, @Thermoking_TE18
    # Approx 3k watts at max hvac
    # thus, a reasonable nominal load is 8k? bottom is 4k?
    XDE35_ESS = op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                       inverter_eff = .97, # @Gallet
                       aux_eff = .89, # Assumption.
                       regen_eff = .6, # Assumption.
                       simple_load = 8000,
                       max_regen = -200000, #w, finally, some good sourcable values.@XDE35_Manual
                       module_struct = (s1,p1),
                       bus_struct = (s2,p2)
                       # W @Siemens_Traction_Motor
                      )
    #print(XDE35_ESS.bus_E_cap()/1000)
    XDE35_ESS.save(ESS_directory + "XDE35_ESS.txt")
    #rint(XDE35_ESS.bus_E_cap())
    return (XDE35_ESS.bus_E_cap()/1000 - 520)**2 / (XDE35_ESS.bus_E_cap()/1000 - XDE35_ESS.bus_E_cap()/1000**2)
    

In [7]:
optimizefunc(find_SP, [(12, 13), (8, 9), (16, 17), (1, 100)])

Running Mirror Point...t...l...


KeyboardInterrupt



KeyboardInterrupt: 

In [18]:
find_SP(12.386325019920218, 7.302245601136856, 16.0, 31.413773426909547)

88.79460849500127

In [19]:
# XDE60 BEV version: 
XDE60 = op.Bus(bus_mass = 20638, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.38, #m, @NF_Xcelsior
               drag_coeff = .7, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XDE60.save(bus_directory + "XDE60.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE60_ESS = op.ESS(motor_eff = .93, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .88, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .54, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000, #w, finally, some good sourcable values.@XDE35_Manual
                   module_struct = (12,8),
                   bus_struct = (16,40)
                   
                   # W @Siemens_Traction_Motor
                  )
XDE60_ESS.save(ESS_directory + "XDE60_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE60_ESS.txt'

In [20]:
# XDE40/XT40 BEV version: 
XDE40 =op.Bus(bus_mass = 13835, #kg, @NF_Xcelsior
                      frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.38, #m, @NF_Xcelsior
               drag_coeff = .7, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )


# Saving the object.
XDE40.save(bus_directory + "XT40.txt")
XDE40.save(bus_directory + "XDE40.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE40_ESS =  op.ESS(motor_eff = .93, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .88, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .54, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000, #w, finally, some good sourcable values.@XDE35_Manual
                   module_struct = (12,8),
                   bus_struct = (16,32)
                   
                   # W @Siemens_Traction_Motor
                  )

XDE40_ESS.save(ESS_directory + "XT40_ESS.txt")
XDE40_ESS.save(ESS_directory + "XDE40_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE40_ESS.txt'

In [21]:
# XDE35 BEV version: 
XDE35 =op.Bus(bus_mass = 13920, #kg, @NF_Xcelsior
                      frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.38, #m, @NF_Xcelsior
               drag_coeff = .7, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )


# Saving the object.

XDE35.save(bus_directory + "XDE35.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE35_ESS =  op.ESS(motor_eff = .93, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .88, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .54, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000, #w, finally, some good sourcable values.@XDE35_Manual
                   module_struct = (12,8),
                   bus_struct = (16,32)
                   
                   # W @Siemens_Traction_Motor
                  )


XDE35_ESS.save(ESS_directory + "XDE35_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE35_ESS.txt'